# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
db.list_collection_names()

['establishments']

In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
where = {'scores.Hygiene': 20}
good_hygiene = establishments.find(where)

# Use count_documents to display the number of documents in the result
print(f"Number of documents: {establishments.count_documents(where)}")
# Display the first document in the results using pprint
pprint(good_hygiene[0])

Number of documents: 41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66761798683d247c76c30c19'),
 'establishments': {'latitude': None, 'longitude': None},
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',

In [7]:
# Convert the result to a Pandas DataFrame
# use json_normalize here to squash the scores and geocode dictionaries
# also use iter() instead of list() to not store the whole dataset in memory
hygiene_df = pd.json_normalize(iter(good_hygiene))
# Display the number of rows in the DataFrame
print(f'Number of rows: {len(hygiene_df)}')
# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

Number of rows: 41


,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,meta.dataSource,meta.extractDate,meta.itemCount,meta.returncode,meta.totalCount,meta.totalPages,meta.pageSize,meta.pageNumber,establishments.latitude,establishments.longitude
0,66761798683d247c76c30c19,110681,0,4029,The Chase Rest Home,Caring Premises,5,5-6 Southfields Road,Eastbourne,East Sussex,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
1,66761798683d247c76c30f95,612039,0,1970/FOOD,Brenalwood,Caring Premises,5,Hall Lane,Walton-on-the-Naze,Essex,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
2,66761798683d247c76c312a5,730933,0,1698/FOOD,Melrose Hotel,Hotel/bed & breakfast/guest house,7842,53 Marine Parade East,Clacton On Sea,Essex,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
3,66761798683d247c76c31497,172953,0,PI/000024532,Golden Palace,Restaurant/Cafe/Canteen,1,5 South Street,Seaford,East Sussex,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
4,66761798683d247c76c314b4,172735,0,PI/000023858,Seaford Pizza,Takeaway/sandwich shop,7844,4 High Street,Seaford,East Sussex,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
5,66761798683d247c76c31e44,512854,0,12/00816/BUTH,Ashby's Butchers,Retailers - other,4613,777 Southchurch Road,Southend-On-Sea,Essex,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
6,66761798683d247c76c32056,1537089,0,22/00224/RESTUN,South Sea Express Cuisine,Restaurant/Cafe/Canteen,1,33 Alexandra Street,Southend-on-sea,Essex,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
7,66761798683d247c76c33584,155648,0,EH/00006058,Golden Palace,Takeaway/sandwich shop,7844,7 London Road,Rayleigh,Essex,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
8,66761798683d247c76c339d3,1012883,0,17/00110/MIXED/S,The Tulip Tree,Restaurant/Cafe/Canteen,1,3 The Village,Chiddingstone,KENT,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
9,66761799683d247c76c341e4,644109,0,41811,F & S,Retailers - other,4613,,81 Southernhay,Basildon,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
where = {'LocalAuthorityName': {'$regex':'London'},'RatingValue': {'$gte':4}}
good_london = establishments.find(where)
# Use count_documents to display the number of documents in the result
print(f"Number of documents: {establishments.count_documents(where)}")
# Display the first document in the results using pprint
pprint(good_london[0])

Number of documents: 33
{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66761798683d247c76c32635'),
 'establishments': {'latitude': None, 'longitude': None},
 'geocode': {'latitude': 51.369321, 'longitude': 0.508551},
 'links': [{'href': 'h

In [9]:
# Convert the result to a Pandas DataFrame
# use json_normalize here to squash the scores and geocode dictionaries
# also use iter() instead of list() to not store the whole dataset in memory
london_df = pd.json_normalize(iter(good_london))

# Display the number of rows in the DataFrame
print(f'Number of rows: {len(london_df)}')

# Display the first 10 rows of the DataFrame
london_df.head(10)

Number of rows: 33


,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,meta.dataSource,meta.extractDate,meta.itemCount,meta.returncode,meta.totalCount,meta.totalPages,meta.pageSize,meta.pageNumber,establishments.latitude,establishments.longitude
0,66761798683d247c76c32635,621707,0,PI/000025307,Charlie's,Other catering premises,7841,Oak Apple Farm Building 103 Sheernes Docks,Sheppy Kent,,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
1,66761798683d247c76c32952,1130836,0,PI/000034075,Mv City Cruises Erasmus,Other catering premises,7841,Cherry Garden Pier,Cherry Garden Street Rotherhithe,Charlton,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
2,66761798683d247c76c334d7,293783,0,PI/000002614,Benfleet Motor Yacht Club,Other catering premises,7841,Ferry Road,South Benfleet Essex,,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
3,66761799683d247c76c342ab,294474,0,PI/000014647,Tilbury Seafarers Centre,Restaurant/Cafe/Canteen,1,Tenants Row Tilbury Docks,Tilbury Essex,,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
4,66761799683d247c76c342ad,1315095,0,PI/000036464,Coombs Catering t/a The Lock and Key,Restaurant/Cafe/Canteen,1,Leslie Ford House,Tilbury,Charlton,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
5,66761799683d247c76c349af,294900,0,PI/000019066,Mv Valulla,Other catering premises,7841,Reeds River Cruises LtdKings ReachRiver Thames...,London,,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
6,66761799683d247c76c36ab5,293756,0,PI/000002538,Tereza Joanne,Other catering premises,7841,Funcraft UK Ltd King George V Dock Woolwich Ma...,London,,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
7,66761799683d247c76c36e9e,293772,0,PI/000002585,The Nuance Group (UK) Limited,Retailers - other,4613,Duty Free Shop Passenger Terminal,London City AirportRoyal DocksLondon,,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
8,66761799683d247c76c36e9f,295107,0,PI/000020749,City Bar & Grill,Restaurant/Cafe/Canteen,1,London City Airport (Airside),London,,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
9,66761799683d247c76c36ea0,294606,0,PI/000016185,WH Smith,Retailers - other,4613,London City Airport,Silvertown London,,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [10]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
penang = establishments.find_one({"BusinessName":"Penang Flavours"})

degree_search = 0.01
#latitude range is .07384 - .09384
#longitude range is 50.490142 to 52.490142
latitude = penang['geocode']['latitude']
longitude = penang['geocode']['longitude']

where = {'geocode.latitude': {'$gte': latitude-degree_search, '$lte':latitude+degree_search},
         'geocode.longitude': {'$gte': longitude-degree_search, '$lte':longitude+degree_search},
         'RatingValue':5}
sort = [('scores.Hygiene', 1)]
limit = 5

# Print the results
pprint(list(establishments.find(where).sort(sort).limit(limit)))

[{'AddressLine1': '130 - 132 Plumstead High Street',
  'AddressLine2': '',
  'AddressLine3': 'Plumstead',
  'AddressLine4': 'Greenwich',
  'BusinessName': 'Volunteer',
  'BusinessType': 'Pub/bar/nightclub',
  'BusinessTypeID': 7843,
  'ChangesByServerID': 0,
  'Distance': 4646.965634598608,
  'FHRSID': 694609,
  'LocalAuthorityBusinessID': 'PI/000116619',
  'LocalAuthorityCode': '511',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'NewRatingPending': False,
  'Phone': '',
  'PostCode': 'SE18 1JQ',
  'RatingDate': '2019-08-05T00:00:00',
  'RatingKey': 'fhrs_5_en-gb',
  'RatingValue': 5,
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('66761799683d247c76c36478'),
  'establishments': {'latitude': None, 'longitude': None},
  'geocode': {'latitude': 51.4873437, 'longitude': 0.09208},
  'links': [{'href': 'http://api.ratings.food.gov.uk/establishments/694609

In [11]:
# Convert result to Pandas DataFrame
top_5_penang_df = pd.json_normalize(iter(establishments.find(where).sort(sort).limit(limit)))
top_5_penang_df.head()

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,meta.dataSource,meta.extractDate,meta.itemCount,meta.returncode,meta.totalCount,meta.totalPages,meta.pageSize,meta.pageNumber,establishments.latitude,establishments.longitude
0,66761799683d247c76c36478,694609,0,PI/000116619,Volunteer,Pub/bar/nightclub,7843,130 - 132 Plumstead High Street,,Plumstead,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
1,66761799683d247c76c36494,694478,0,PI/000086506,Atlantic Fish Bar,Takeaway/sandwich shop,7844,35 Lakedale Road,,Plumstead,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
2,66761799683d247c76c364a6,909912,0,PI/000201100,Lumbini Grocery Ltd T/A Al-Iman,Retailers - other,4613,17 Lakedale Road,,Plumstead,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
3,66761799683d247c76c3644b,695223,0,PI/000178842,Iceland,Retailers - supermarkets/hypermarkets,7840,144 - 146 Plumstead High Street,,Plumstead,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None
4,66761799683d247c76c36457,1380578,0,14425,Howe and Co Fish and Chips - Van 17,Mobile caterer,7846,Restaurant And Premises 107A Plumstead High St...,,Plumstead,...,None,0001-01-01T00:00:00,0,None,0,0,0,0,None,None


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [15]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
match = {'$match': {'scores.Hygiene':0}}
# 2. Groups the matches by Local Authority
group = {'$group': {'_id':'$LocalAuthorityName', 'count': { '$sum': 1 }}}
# 3. Sorts the matches from highest to lowest
sort = {'$sort': {'count': -1, 'LocalAuthorityName':1}}

pipeline=[match, group, sort]
# Print the number of documents in the result
results = list(establishments.aggregate(pipeline))
# Print the first 10 results
pprint(results[:10])

[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [17]:
# Convert the result to a Pandas DataFrame
bad_hygiene_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print(f'Rows in DataFrame: {len(bad_hygiene_df)}')
# Display the first 10 rows of the DataFrame
bad_hygiene_df.head(10)

Rows in DataFrame: 55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
